In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.cuda.is_available()

True

In [8]:
class LoRaLayer(nn.Module):
    def __init__(self, input_dim, output_dim, rank):
        super(LoRaLayer, self).__init__()
        self.U = nn.Parameter(torch.randn(input_dim, rank))
        self.V = nn.Parameter(torch.randn(rank, output_dim))
        # 대각 요소만 학습 가능하게 설정
        self.S_values = nn.Parameter(torch.ones(rank))  # 초기값을 1로 설정

    def forward(self, x):
        # 학습 가능한 대각 요소를 이용해 대각 행렬 S 생성
        S = torch.diag(self.S_values)
        weight = torch.mm(self.U, torch.mm(S, self.V))
        return torch.mm(x, weight)
    
lr_layer = LoRaLayer(64, 64, 30)
lr_layer.S_values

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], requires_grad=True)

In [ ]:
class HyperNetwork(nn.Module):
    def __init__(self, num_coefficients, num_layers, rank):
        super(HyperNetwork, self).__init__()
        self.num_layers = num_layers
        self.rank = rank
        # HyperNetwork는 PDE 계수에 따라 각 LoRaLayer의 S 대각 요소를 생성
        self.fc_hyper = nn.Linear(num_coefficients, num_layers * rank)
    
    def forward(self, coefficients):
        # coefficients는 PDE의 계수
        diag_elements = self.fc_hyper(coefficients)
        # 각 LoRaLayer에 대한 대각 요소를 반환
        return diag_elements.view(self.num_layers, self.rank)